# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [2]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [3]:
import pandas as pd
import numpy as np
import re
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [4]:
# Path hack to allow imports from the parent directory.
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

In [5]:
from isadream.isadream.models import utils
from isadream.isadream import io

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [6]:
# A demo json file is provided.
nmr_json_demo = utils.SIPOS_DEMO
demo_base_path = utils.BASE_PATH
print(nmr_json_demo, '\n', demo_base_path)

/home/tylerbiggs/git/isadream/isadream/demo_data/demo_json/sipos_2006_talanta_nmr_figs.json 
 /home/tylerbiggs/git/isadream/isadream/demo_data/


---

In [35]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_json(demo_json)

test_groups = (
    ('27 Al ppm', 'ppm', ("Al",)),
    ('Total Aluminate Concentration', 'Molar', ("Al", )),
    ('Counter Ion Concentration', 'Molar', ("Na+", "Li+", "Cs+", "K+", )),
    ('Counter Ion', 'Species', ("Na+", "Li+", "Cs+", "K+", )),
    ('Base Concentration', 'Molar', ("OH-", )),
)

node_assays = [a.build_column_data_dict(test_groups) for a in node.assays]

dfs = [pd.DataFrame(cds) for cds in node_assays]
df = pd.concat(dfs)
df

,27 Al ppm,Total Aluminate Concentration,Counter Ion Concentration,Counter Ion,Base Concentration
0,79.90,0.005,2.93,{K+},2.93
1,79.84,0.005,4.92,{K+},4.92
2,79.72,0.005,6.85,{K+},6.85
3,79.66,0.005,9.13,{K+},9.13
4,79.66,0.005,10.71,{K+},10.71
0,79.92,0.005,0.66,{Li+},0.66
1,79.84,0.005,1.10,{Li+},1.10
2,79.66,0.005,1.64,{Li+},1.64
3,79.54,0.005,2.14,{Li+},2.14
4,79.36,0.005,2.59,{Li+},2.59


---